In [1]:
import torch
import numpy as np
import segmentation_models_pytorch as smp

import torch.nn as nn
from torchmetrics.functional import accuracy

import pytorch_lightning as pl
from pytorch_lightning.callbacks import EarlyStopping, ModelCheckpoint
from pytorch_lightning.loggers import WandbLogger

from torchlightning_module import torch_lightning_DataModule
from dataset_module import DatasetModule

#torch.set_float32_matmul_precision('medium')

###
class SemanticSegmentationModel(pl.LightningModule):
    def __init__(self, hparams):
        super().__init__()
        for key in hparams.keys():
             self.hparams[key]=hparams[key]

        self.loss_fn = nn.CrossEntropyLoss()
        self.save_hyperparameters()
        
        self.model = smp.FPN( #modifiy in-channels and output. 
           encoder_name=self.hparams['encoder_name'], 
           classes=self.hparams['classes'], 
           activation=self.hparams['activation'], 
           encoder_weights=self.hparams['encoder_weights'],
            in_channels=self.hparams['in_channels'],

              )

    def forward(self, x):
        return self.model(x)

    def training_step(self, batch, batch_idx):
        images, labels = batch
        outputs = self.model(images)
        labels = labels.squeeze(1).long() 
        loss = self.loss_fn(outputs, labels)
        
        
        #outputs : output of model > logits /// labels > y ///images > x 
        self.log('train_loss',loss, on_step=False, on_epoch=True)
        return loss
    
    
#     def training_epoch_end(self, outputs):
#         avg_loss = torch.stack([x["loss"] for x in outputs]).mean()
#         self.log("avg_train_loss", avg_loss)
#         wandb.log({"avg_train_loss": avg_loss})


    def validation_step(self, batch, batch_idx):
        images, labels = batch
        outputs = self.model(images)
        labels = labels.squeeze(1).long()
        #print(f'{labels.shape} is shape of label in the val step method')
        #print(f'{outputs.shape} is shape of outputs in the val step method')

        loss = self.loss_fn(outputs, labels)
        # Log loss and accuracy
        self.log('val_loss', loss, on_step=False, on_epoch=True)
        
        return {'val_loss': loss}

#     def validation_epoch_end(self, outputs):
#         avg_loss = torch.stack([x['val_loss'] for x in outputs]).mean()
#         logs = {'val_loss': avg_loss}
#         return {'val_loss': avg_loss, 'log': logs}

        # def validation_epoch_end(self, outputs):
        #     avg_loss = torch.stack([x['val_loss'] for x in outputs]).mean()
        #     logs = {'val_loss': avg_loss}
        #     self.log('val_loss', avg_loss)



    def configure_optimizers(self):
        return torch.optim.Adam(self.parameters(), lr=self.hparams.lr)

    
# def lr_schedule(step):
#    lr = 0.001
#    if step < 10:
#        return lr
#    elif step < 20:
#        return lr / 2
#    else:
#        return lr / 4

#lr_scheduler = pl.callbacks.LearningRateScheduler(lr_schedule)

early_stopping = EarlyStopping(
    monitor='val_loss',
    patience=5,
    mode='min'
)


# Define your hyperparameters
hparams = {
    'encoder_name': 'resnet34', #resnet50
    'classes': 19,
    'activation': 'softmax',
    'encoder_weights': 'imagenet', #imagenet
    'lr': 0.001,#  
    'in_channels':5,

}

# Initialize your model
model = SemanticSegmentationModel(hparams)
data=torch_lightning_DataModule()

wandb_logger = WandbLogger(project='phase2_semantic_segmentation_initial_run')

trainer = pl.Trainer(max_epochs=10,
                     accelerator='gpu',
                     callbacks=[early_stopping],
                     logger=wandb_logger
                    ) #lr_scheduler

trainer.fit(model,data)

/opt/conda/envs/venv/lib/python3.9/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: markalsa. Use `wandb login --relogin` to force relogin


GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name    | Type             | Params
---------------------------------------------
0 | loss_fn | CrossEntropyLoss | 0     
1 | model   | FPN              | 23.2 M
---------------------------------------------
23.2 M    Trainable params
0         Non-trainable params
23.2 M    Total params
92.656    Total estimated model params size (MB)


Sanity Checking: 0it [00:00, ?it/s]

/opt/conda/envs/venv/lib/python3.9/site-packages/pytorch_lightning/trainer/connectors/data_connector.py:224: PossibleUserWarning: The dataloader, val_dataloader 0, does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` (try 64 which is the number of cpus on this machine) in the `DataLoader` init to improve performance.
  rank_zero_warn(


Sanity Checking DataLoader 0:  50%|█████     | 1/2 [00:00<00:00,  1.04it/s]

/opt/conda/envs/venv/lib/python3.9/site-packages/segmentation_models_pytorch/base/modules.py:116: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  return self.activation(x)


/opt/conda/envs/venv/lib/python3.9/site-packages/pytorch_lightning/trainer/connectors/data_connector.py:224: PossibleUserWarning: The dataloader, train_dataloader, does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` (try 64 which is the number of cpus on this machine) in the `DataLoader` init to improve performance.
  rank_zero_warn(
/opt/conda/envs/venv/lib/python3.9/site-packages/pytorch_lightning/trainer/trainer.py:1609: PossibleUserWarning: The number of training batches (5) is smaller than the logging interval Trainer(log_every_n_steps=50). Set a lower value for log_every_n_steps if you want to see logs for the training epoch.
  rank_zero_warn(


Epoch 0:  71%|███████▏  | 5/7 [00:05<00:02,  1.13s/it, loss=2.22, v_num=7tz2]
Validation: 0it [00:00, ?it/s]
Epoch 1:  71%|███████▏  | 5/7 [00:05<00:02,  1.06s/it, loss=2.13, v_num=7tz2]
Validation: 0it [00:00, ?it/s]
Epoch 2:  71%|███████▏  | 5/7 [00:05<00:02,  1.06s/it, loss=2.1, v_num=7tz2] 
Validation: 0it [00:00, ?it/s]
Epoch 3:  71%|███████▏  | 5/7 [00:05<00:02,  1.06s/it, loss=2.08, v_num=7tz2]
Validation: 0it [00:00, ?it/s]
Epoch 4:  71%|███████▏  | 5/7 [00:05<00:02,  1.10s/it, loss=2.03, v_num=7tz2]
Validation: 0it [00:00, ?it/s]
Epoch 5:  71%|███████▏  | 5/7 [00:05<00:02,  1.11s/it, loss=2.03, v_num=7tz2]
Validation: 0it [00:00, ?it/s]
Epoch 6:  71%|███████▏  | 5/7 [00:05<00:02,  1.10s/it, loss=2.03, v_num=7tz2]
Validation: 0it [00:00, ?it/s]
Epoch 7:  57%|█████▋    | 4/7 [00:04<00:03,  1.04s/it, loss=2.03, v_num=7tz2]

/opt/conda/envs/venv/lib/python3.9/site-packages/pytorch_lightning/trainer/call.py:48: UserWarning: Detected KeyboardInterrupt, attempting graceful shutdown...
  rank_zero_warn("Detected KeyboardInterrupt, attempting graceful shutdown...")


In [1]:
import multiprocessing

num_workers = multiprocessing.cpu_count() - 1


In [3]:
num_workers

31